In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score


In [ ]:
csv_file_path = "datar.csv"
data = pd.read_csv(csv_file_path)

In [ ]:
data

In [ ]:
data['odor_labels_filtered'] = data['odor_labels_filtered'].apply(eval)

In [ ]:
mlb = MultiLabelBinarizer()
labels_binary_matrix = mlb.fit_transform(data['odor_labels_filtered'])
labels_df = pd.DataFrame(labels_binary_matrix, columns=mlb.classes_)

In [ ]:
X_train = data[data['labels_train/test'] == 1].drop(['odor_labels_filtered', 'labels_train/test'], axis=1).iloc[:,2:]
X_test = data[data['labels_train/test'] == 0].drop(['odor_labels_filtered', 'labels_train/test'], axis=1).iloc[:,2:]

y_train = labels_df[data['labels_train/test'] == 1]
y_test = labels_df[data['labels_train/test'] == 0]


In [ ]:
X_train_cleaned = X_train.dropna()
X_test_cleaned = X_test.dropna()

dropped_indices_train = X_train.index.difference(X_train_cleaned.index)
dropped_indices_test = X_test.index.difference(X_test_cleaned.index)

y_train_cleaned = y_train.drop(index=dropped_indices_train)
y_test_cleaned = y_test.drop(index=dropped_indices_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import numpy as np

# This was chosen random and needs to be optimized
rf = RandomForestClassifier(
    n_estimators=1000,
    max_depth=30,
    min_samples_split=4,
    min_samples_leaf=1,
    max_features='sqrt',
    class_weight='balanced',
    random_state=42
)


rf.fit(X_train_cleaned, y_train_cleaned)

y_pred_proba = rf.predict_proba(X_test_cleaned)


y_pred_proba = np.array([proba[:, 1] for proba in y_pred_proba]).T

auc_scores = []
for i in range(y_test_cleaned.shape[1]):
    auc_score = roc_auc_score(y_test_cleaned.iloc[:, i], y_pred_proba[:, i])
    auc_scores.append(auc_score)
    print(f"AUROC for {y_test_cleaned.columns[i]}: {auc_score}")

avg_auc = np.mean(auc_scores)
print(f"Average AUROC: {avg_auc}")
